## Tik Tok

In [1]:
import os
import numpy as np
from py2neo import Graph
from py2neo import Node
from py2neo import Relationship
import pandas as pd
import numpy as np
# for some nlp secret sauce
import nltk
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from langdetect import detect_langs
from langdetect import detect
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
# Run the next two lines as needed
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\arual\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arual\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# update bolt port number and password as needed.
# in Neo4j, start DB of choice, click on DB name, on right under Details tab note 'Bolt port'
# Windows
if os.name == 'nt':
    graph = Graph("bolt://localhost:7687", password='1234', name='neo4j')
# Linux & Mac
if os.name == 'posix':
    graph = Graph("bolt://localhost:11003", password='xxx', name='neo4j')

In [5]:
# I don't think this needs to be run every time
query = """
CREATE INDEX USRIDX IF NOT EXISTS
FOR (n:user)
ON (n.id);
"""
graph.run(query)

(No data)

In [6]:
# Quick test to make sure things are connected and working. This Okean Elzy song should show up twice.
query = """
MATCH (s:song) WHERE s.title =~ '.*Обійми.*' RETURN s;
"""
graph.run(query)

s
"(_342470:song {album: '\u0417\u0435\u043c\u043b\u044f', audioURLWithCookie: 0, authorName: '\u041e\u043a\u0435\u0430\u043d \u0415\u043b\u044c\u0437\u0438', coverLarge: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/0e057b52e54948e8a5d6ebc686ef8e3b~c5_720x720.jpeg', coverMedium: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/0e057b52e54948e8a5d6ebc686ef8e3b~c5_200x200.jpeg', coverThumb: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/0e057b52e54948e8a5d6ebc686ef8e3b~c5_100x100.jpeg', duration: 60, id: '6718412074684254210', keyToken: '', numVids: 19300, original: 0, playUrl: 'https://sf16-ies-music-va.tiktokcdn.com/obj/tos-useast2a-ve-2774/76156347b51b4cbaa836c9e444e5872b', private: 0, scheduleSearchTime: 0, title: '\u041e\u0431\u0456\u0439\u043c\u0438'})"
"(_343278:song {album: '\u041e\u0431\u0456\u0439\u043c\u0438 - Remix', audioURLWithCookie: 0, authorName: '\u041e\u043a\u0435\u0430\u043d \u0415\u043b\u044c\u0437\u0438', coverLarge: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/60a559e85efd4e40877e660f651efdfd~c5_720x720.jpeg', coverMedium: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/60a559e85efd4e40877e660f651efdfd~c5_200x200.jpeg', coverThumb: 'https://p16-amd-va.tiktokcdn.com/img/tos-useast2a-v-2774/60a559e85efd4e40877e660f651efdfd~c5_100x100.jpeg', duration: 60, id: '6899164801193216002', keyToken: '', numVids: '6733', original: 0, playUrl: 'https://sf16-ies-music-va.tiktokcdn.com/obj/tos-useast2a-ve-2774/f2e8a7defbd44b08858d6c6d04eefc97', private: 0, scheduleSearchTime: 0, title: '\u041e\u0431\u0456\u0439\u043c\u0438 - Remix'})"


In [10]:
#Most liked accounts related to news (not just official media )

query = """
MATCH (u:user)-[:author]->(v:video)
WHERE u.id =~ '.*new.*'
RETURN COLLECT(DISTINCT u.id) AS userName, u.verified as VerifiedAccount , u.heartCount as LikesReceived
ORDER BY LikesReceived DESC 
"""
lista=pd.DataFrame(graph.run(query).data())
lista


,userName,VerifiedAccount,LikesReceived
0,[cbsnews],1,153400000
1,[yahoonews],1,97800000
2,[nbcnews],1,79200000
3,[underthedesknews],1,76600000
4,[nikocadodailynews],0,71100000
...,...,...,...
67,[thegossipnewstribune],0,23400
68,[forcesnews],0,15600
69,[ykrainews],0,8886
70,[military_newss],0,5307


In [11]:
#Most followed account with any relation with media (We can see how in the top 10 we alredy have accounts that doesn't have anything to do with the conflict)
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE u.id =~ '.*new.*'
RETURN COLLECT(DISTINCT u.id) AS userName, u.verified as VerifiedAccount , u.followerCount as Followers
ORDER BY Followers DESC ;
"""
lista=pd.DataFrame(graph.run(query).data())
lista

,userName,VerifiedAccount,Followers
0,[nbcnews],1,2700000
1,[cbsnews],1,2500000
2,[underthedesknews],1,2300000
3,[abcnews],1,2100000
4,"[yahoonews, .thenewwave]",1,1700000
...,...,...,...
69,[world_news_clips],0,2203
70,[ykrainews],0,1198
71,[thegossipnewstribune],0,966
72,[military_newss],0,922


In [11]:
#Most followed account with any relation with media (We can see how in the top 10 we alredy have accounts that doesn't have anything to do with the conflict)
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE u.id =~ '.*new.*' and 
v.desc =~ '.*war.*'
RETURN COLLECT(DISTINCT u.id) AS userName, u.verified as VerifiedAccount , u.followerCount as Followers
ORDER BY Followers DESC ;
"""
results1=pd.DataFrame(graph.run(query).data())

In [12]:
results1

,userName,VerifiedAccount,Followers
0,[nbcnews],1,2700000
1,[cbsnews],1,2500000
2,[donald_newshd],0,1400000
3,[viceworldnews],1,1300000
4,"[skynews, goodmorningbadnews]",1,1100000
5,[bikini.bottom.news],0,1100000
6,[world247breakingnews],0,397400
7,[btnewsroom],0,320000
8,[quicktakenews],1,189800
9,[caspolnews],0,128900


In [13]:
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE u.id =~ '.*new.*' and 
v.desc =~ '.*ukraine.*'
RETURN COLLECT(DISTINCT u.id) AS userName, u.verified as VerifiedAccount , u.followerCount as Followers
ORDER BY Followers DESC ;
"""
results2=pd.DataFrame(graph.run(query).data())

In [14]:
results2

,userName,VerifiedAccount,Followers
0,[cbsnews],1,2500000
1,[viceworldnews],1,1300000
2,"[skynews, goodmorningbadnews]",1,1100000
3,[bikini.bottom.news],0,1100000
4,[news.com.au],1,822800
5,[world247breakingnews],0,397400
6,[caspolnews],0,128900
7,[official__world_news],0,48300
8,[jayhawkdlonewolf],0,47400
9,[bbcnewsukrainian],1,36000


In [15]:
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE u.id =~ '.*new.*' and 
v.desc =~ '.*russia.*'
RETURN COLLECT(DISTINCT u.id) AS userName, u.verified as VerifiedAccount , u.followerCount as Followers
ORDER BY Followers DESC ;
"""
results3=pd.DataFrame(graph.run(query).data())
results3

,userName,VerifiedAccount,Followers
0,[cbsnews],1,2500000
1,[donald_newshd],0,1400000
2,[viceworldnews],1,1300000
3,"[skynews, goodmorningbadnews]",1,1100000
4,[bikini.bottom.news],0,1100000
5,[news.com.au],1,822800
6,[world247breakingnews],0,397400
7,[caspolnews],0,128900
8,[factualnews77],0,66900
9,[official__world_news],0,48300


In [16]:
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE u.id =~ '.*new.*' and 
v.desc =~ '.*conflict.*'
RETURN COLLECT(DISTINCT u.id) AS userName, u.verified as VerifiedAccount , u.followerCount as Followers
ORDER BY Followers DESC ;
"""
results4=pd.DataFrame(graph.run(query).data())
results4

,userName,VerifiedAccount,Followers
0,[cbsnews],1,2500000
1,[skynews],1,1100000
2,[caspolnews],0,128900
3,[latestconflictwarnews2],0,21600
4,[thenewsmovement],1,20600
5,[forcesnews],0,2633
6,[world_news_clips],0,2203
7,[thegossipnewstribune],0,966
8,[military_newss],0,922


Maybe this last one is too small to look for a intersection, so lest just pick the first three 

In [28]:
#If you check for the suggested acounts for the most popular news accounts, it doesn't really relate you to just new's content.
#Maybe it'll be good to implement a loop to look for the intersection among the first ten to see if TIKTOK is trying to 
#share just the same accounts under his algorithms.
#What would we understand as ukraine/russia related?

The following Data Frames sort the accounts by streams, filtering by mentioning the war, conflict, ukraine and russia.

In [12]:
#War
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE (u.id =~ '.*news.*' OR
      v.desc =~ '.*news.*' OR
      v.desc =~ '.*новини.*' OR 
      v.desc =~ '.*Новости.*') AND 
      (v.desc =~ '.*war.*' OR
      v.desc =~ '.*війни.*' OR
      v.desc =~ '.*война.*')
RETURN  u.id AS userName, u.verified as VerifiedAccount,count(v) as VidRelated,u.videoCount as Totalvid,sum(v.playCount) as Streams
ORDER BY Streams DESC ;
"""
war=pd.DataFrame(graph.run(query).data())
war

,userName,VerifiedAccount,VidRelated,Totalvid,Streams
0,bikini.bottom.news,0,12,39,36651600
1,skynews,1,18,738,33906575
2,cheddygrace,0,1,767,31600000
3,president.putin.fanpage,0,15,242,30406300
4,authentictimetraveler,0,7,112,27760800
...,...,...,...,...,...
119,wanhedaa,0,1,141,2042
120,karlemmrich,0,1,1376,1688
121,militaryforces.com,0,1,156,1570
122,ivxxuk,0,1,143,1206


In [13]:
war["VerifiedAccount"].sum()

19

In [103]:
#proportion of verified accounts
19/124

0.1532258064516129

In [101]:
users1=war["userName"].to_list()


In [15]:
#Ukraine
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE (u.id =~ '.*news.*' OR
      v.desc =~ '.*news.*' OR
      v.desc =~ '.*новини.*' OR 
      v.desc =~ '.*Новости.*') AND 
      (v.desc =~ '.*ukraine.*' OR
      v.desc =~ '.*україни.*'OR
      v.desc =~ '.*Украина.*')
RETURN  u.id AS userName, u.verified as VerifiedAccount,count(v) as VidRelated,u.videoCount as Totalvid,sum(v.playCount) as Streams
ORDER BY Streams DESC ;
"""
ukraine=pd.DataFrame(graph.run(query).data())
ukraine

,userName,VerifiedAccount,VidRelated,Totalvid,Streams
0,megatmurad,0,15,32,39641600
1,bikini.bottom.news,0,14,39,37147500
2,skynews,1,2,738,32625800
3,cheddygrace,0,1,767,31600000
4,president.putin.fanpage,0,16,242,30492700
...,...,...,...,...,...
125,r3dwyn,0,1,31,1974
126,militaryforces.com,0,1,156,1570
127,linkyfirsttiktok,0,2,114,1214
128,thealetheiaproject,0,1,15,1092


In [16]:
ukraine["VerifiedAccount"].sum()

18

In [104]:
#proportion of verified accounts
18/130

0.13846153846153847

In [100]:
users2=ukraine["userName"].to_list()


In [18]:
#Russia
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE (u.id =~ '.*news.*' OR
      v.desc =~ '.*news.*' OR
      v.desc =~ '.*новини.*'  OR 
      v.desc =~ '.*Новости.*') AND 
      (v.desc =~ '.*russia.*' OR
      v.desc =~ '.*Росія.*' OR
      v.desc =~ '.*Россия.*')
RETURN  u.id AS userName, u.verified as VerifiedAccount,count(v) as VidRelated,u.videoCount as Totalvid,sum(v.playCount) as Streams
ORDER BY Streams DESC ;
"""
russia=pd.DataFrame(graph.run(query).data())
russia

,userName,VerifiedAccount,VidRelated,Totalvid,Streams
0,president.putin.fanpage,0,16,242,39692700
1,megatmurad,0,15,32,39641600
2,bikini.bottom.news,0,14,39,37147500
3,skynews,1,3,738,32649600
4,cheddygrace,0,1,767,31600000
...,...,...,...,...,...
120,militaryforces.com,0,1,156,1570
121,linkyfirsttiktok,0,2,114,1214
122,thealetheiaproject,0,1,15,1092
123,ykrainews,0,1,16,408


In [19]:
russia["VerifiedAccount"].sum()

18

In [105]:
#proportion of verified accounts
18/125

0.144

In [99]:
users3=russia["userName"].to_list()


In [21]:
#Conflict
query = """
MATCH (u:user)-[:author]->(v:video)
WHERE (u.id =~ '.*new.*' OR
      v.desc =~ '.*news.*' OR
      v.desc =~ '.*новини.*'  OR 
      v.desc =~ '.*Новости.*') AND 
      (v.desc =~ '.*conflict.*' OR
      v.desc =~ '.*конфлікт.*' OR
      v.desc =~ '.*конфликт.*')
RETURN  u.id AS userName, u.verified as VerifiedAccount,count(v) as VidRelated,u.videoCount as Totalvid,sum(v.playCount) as Streams
ORDER BY Streams DESC ;
"""
conflict=pd.DataFrame(graph.run(query).data())
conflict

,userName,VerifiedAccount,VidRelated,Totalvid,Streams
0,caspolnews,0,3,404,9341700
1,therealisticfishhead,0,1,90,5300000
2,themsinstem,0,15,339,5105004
3,latestconflictwarnews2,0,5,98,1338500
4,lordeedge,0,2,1348,993200
5,fundiambb1,0,1,481,706700
6,thegossipnewstribune,0,27,32,532612
7,world_news_clips,0,2,26,347287
8,project_bandito,0,27,44,323462
9,wearescotland,0,1,740,306500


In [22]:
conflict["VerifiedAccount"].sum()

4

In [106]:
#proportion of verified accounts
4/31

0.12903225806451613

In [96]:
#Scoring each account by the descriptions of their videos related to the war, ukraine and russia
sai = SentimentIntensityAnalyzer()
news=conflict["userName"]
for index, new in enumerate(news):
    score=[]
    query = """
    MATCH (u:user)-[a:author]->(v:video)
    WHERE u.id =~ '%s' and
          (v.desc =~ '.*conflict.*' OR
      v.desc =~ '.*конфлікт.*' OR
      v.desc =~ '.*конфликт.*')
    RETURN v.desc AS description; 
    """ % new
    # the % new inserts value of new where the %s is in the WHERE statement
    result=pd.DataFrame(graph.run(query).data())
    if (len(result)!=0):
        for sentence in result['description']:
            lang = detect(sentence)
            if lang=='en':
                kvp = sai.polarity_scores(sentence)
                score.append(kvp['compound'])
        #print(new,'has mean sentiment score of', round(np.mean(score),3),'from', len(score),'videos')
        #print()
        conflict.at[index, "Score"]=round(np.mean(score),3)
        
    else:
        print(new,'returned no results in video description')
        print()

c:\users\arual\appdata\local\programs\python\python39\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\arual\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\users\arual\appdata\local\programs\python\python39\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\users\arual\appdata\local\programs\python\python39\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [113]:
conflict

,userName,VerifiedAccount,VidRelated,Totalvid,Streams,Score
0,caspolnews,0,3,404,9341700,-0.127
1,therealisticfishhead,0,1,90,5300000,-0.318
2,themsinstem,0,15,339,5105004,0.000
3,latestconflictwarnews2,0,5,98,1338500,-0.260
4,lordeedge,0,2,1348,993200,0.000
5,fundiambb1,0,1,481,706700,0.000
6,thegossipnewstribune,0,27,32,532612,0.000
7,world_news_clips,0,2,26,347287,0.000
8,project_bandito,0,27,44,323462,-0.015
9,wearescotland,0,1,740,306500,0.000


In [114]:
users4=conflict["userName"].to_list()


In [98]:
conflict["Score"].mean()

-0.12010344827586207

es practicamente el mismo porcentaje que en los demás casos, simplemente es una cuarta parte de los datos recogidos en los demás casos :/
Conflict seems to be the less used word, so we'll study it apart, to keep the numbers compensate. 
Let's study a little the type of messages comming from them. Descriptions are also condening the war with negative sensitivity.

In [115]:
#Intersection of the different list of accounts (war/ukraine/russia)
users=list(set(users1) & set(users2)& set(users3))

#Including the russian terminology doesn't affect the number of any list

prueba=list(set(users1) & set(users2)& set(users3) & set(users4))

In [117]:
prueba

['latestconflictwarnews2',
 'skynews',
 'therealisticfishhead',
 'lordeedge',
 'supercad55',
 'robertjackcutter',
 'dailyrundownwithchris',
 'theverymessyworld',
 'fundiambb1',
 'city__resident',
 'thenewsmovement',
 'thegossipnewstribune',
 'cnn',
 'howya.now',
 'relevanttopics',
 'themsinstem',
 'project_bandito',
 'cbsnews',
 'tiktok__czech',
 'military_newss',
 'countdaedalus',
 'ca_astronorthstarph',
 'crazyniiiik',
 'thedailyrundown',
 'seoulsecrets',
 'forcesnews',
 'caspolnews',
 'world_news_clips',
 'cazacaza23']

Podemos concluir ya de antemano, que tiktok no es una plataforma muy usada por medios tradicionales, ya que en nuestros filtros, el número demedios verificados no superan el 15% en ninguno de los casos 

In [ ]:
#should we filter by the visualizations of the war-related content or the general stream of the content? 
#That's just changing the WHERE quote

In [85]:
#First we will sort the list of the users by streaming again and we will select the main 20
query=  """
WITH ['president.putin.fanpage','frankdomenic','random.content99','bbcnewsukrainian','ykrainews','joy_enjoy6','themsinstem',
'project_bandito','ervesto77','about_ww3','tiktok__czech','picklenews','visegrad24','newswoow','abc15arizona','dogedave',
 'tip_berlin','ukrainerussianewsdaily','papakreem','militaryforces.com','dark_engine84','whitecheddarbread','caspolnews',
 'tv13news','world_news_clips','cbsmornings','crease19','lordeedge','dailyrundownwithchris','theverymessyworld','fundiambb1',
 'viceworldnews','aaronparnas6','internationalnews12365','bigmike3223','cheddygrace','cbsnews','matthewcassel','seoulsecrets',
 'world247breakingnews','an4e216','latestconflictwarnews2','military_thingz','goodmorningbadnews','supercad55','robertjackcutter',
 'aviationhxb','morriganamber','thenewsmovement','belgian_warrior_group','relevanttopics','stand.with.ukraine.now','iam.rob21',
 'david.heath','military_newss','dailymail','xenasolo','gregmustreader','eurovision.by.kseniia','ca_astronorthstarph',
 'crazyniiiik','planefinder','thealetheiaproject','ukraine.news.today','the_ghost_ace_of_kiev','forcesnews',
 'newz2you','bbcnews','s1xrey','.military.newss','highlightsideline','skynews','therealisticfishhead','_rachtok_','city__resident',
 'u24_news','thegossipnewstribune','cnn','sashareheylo','howya.now','politicsoftheworld','snoopy.vn','frankorando','mihhail83',
 'thefirstnews','healingwithirina','themiracle35','thehistorylegends','astridszarek','countdaedalus','ukraine_latest','thedailyrundown',
 'christina_hrystya','bikini.bottom.news','ukraine_war_1','bigkingpiz','zeitimbild','1newsnz','cazacaza23'] as users
MATCH (u:user)-[:author]->(v:video)
WHERE u.id in users and
          (v.desc =~ '.*ukraine.*' OR
          v.desc =~ '.*war.*' OR 
          v.desc =~ '.*russia.*')
RETURN   u.id AS userName, u.verified as VerifiedAccount , sum(v.playCount) as Streams
ORDER BY Streams DESC ;
""" 
sortedusers=pd.DataFrame(graph.run(query).data()).head(20)
sortedusers

,userName,VerifiedAccount,Streams
0,president.putin.fanpage,0,48940200
1,viceworldnews,1,41258500
2,bikini.bottom.news,0,37147500
3,skynews,1,34132375
4,cheddygrace,0,31600000
5,xenasolo,0,30629200
6,cbsnews,1,24747400
7,world247breakingnews,0,24664271
8,abc15arizona,1,23950384
9,matthewcassel,0,23114400


In [86]:
sortedusers["VerifiedAccount"].sum()
#We can see how is less that the 50% of the most consumed accounts 
#If we filter the ussers by the videos it drops 1 unit

6

In [108]:
sortedusers["userName"].to_list()

['president.putin.fanpage',
 'viceworldnews',
 'bikini.bottom.news',
 'skynews',
 'cheddygrace',
 'xenasolo',
 'cbsnews',
 'world247breakingnews',
 'abc15arizona',
 'matthewcassel',
 'aaronparnas6',
 'planefinder',
 'mihhail83',
 'military_thingz',
 'visegrad24',
 'random.content99',
 'politicsoftheworld',
 'aviationhxb',
 'cbsmornings',
 'city__resident']

In [87]:
#Scoring each account by the descriptions of their videos related to the war, ukraine and russia
sai = SentimentIntensityAnalyzer()
news=sortedusers["userName"]
for index, new in enumerate(news):
    score=[]
    query = """
    MATCH (u:user)-[a:author]->(v:video)
    WHERE u.id =~ '%s' and
          (v.desc =~ '.*ukraine.*' OR
          v.desc =~ '.*war.*' OR 
          v.desc =~ '.*russia.*')
    RETURN v.desc AS description; 
    """ % new
    # the % new inserts value of new where the %s is in the WHERE statement
    result=pd.DataFrame(graph.run(query).data())
    if (len(result)!=0):
        for sentence in result['description']:
            lang = detect(sentence)
            if lang=='en':
                kvp = sai.polarity_scores(sentence)
                score.append(kvp['compound'])
        #print(new,'has mean sentiment score of', round(np.mean(score),3),'from', len(score),'videos')
        #print()
        sortedusers.at[index, "Score"]=round(np.mean(score),3)
        
    else:
        print(new,'returned no results in video description')
        print()

In [79]:
####PRUEBA#######
##No compilar hasta saber como cambiar los valures null###
#Scoring each account by the descriptions and stickers of their videos related to the war, ukraine and russia
#Como lidiar con los missing stickers? "null"
sai = SentimentIntensityAnalyzer()
news=sortedusers["userName"]
for index, new in enumerate(news):
    score=[]
    query = """
    MATCH (u:user)-[a:author]->(v:video)
    WHERE u.id =~ '%s' and
          (v.desc =~ '.*ukraine.*' OR
          v.desc =~ '.*war.*' OR 
          v.desc =~ '.*russia.*')
    RETURN v.desc AS description, v.stickerText as sticker; 
    """ % new
    # the % new inserts value of new where the %s is in the WHERE statement
    result=pd.DataFrame(graph.run(query).data())
    if (len(result)!=0):
        for sentence in result['description']:
            lang = detect(sentence)
            if lang=='en':
                kvp = sai.polarity_scores(sentence)
                score.append(kvp['compound'])
        #print(new,'has mean sentiment score of', round(np.mean(score),3),'from', len(score),'videos')
        #print()
        sortedusers.at[index, "Description score"]=round(np.mean(score),3)
        for sentence in result['sticker']:
            lang = detect(sentence)
            if lang=='en':
                kvp = sai.polarity_scores(sentence)
                score.append(kvp['compound'])
        #print(new,'has mean sentiment score of', round(np.mean(score),3),'from', len(score),'videos')
        #print()
        sortedusers.at[index, "Sticker score"]=round(np.mean(score),3)
        
    else:
        print(new,'returned no results in video description or sticker')
        print()
        
    

TypeError: expected string or bytes-like object

In [88]:
sortedusers

,userName,VerifiedAccount,Streams,Score
0,president.putin.fanpage,0,48940200,0.114
1,viceworldnews,1,41258500,-0.263
2,bikini.bottom.news,0,37147500,-0.053
3,skynews,1,34132375,-0.173
4,cheddygrace,0,31600000,-0.511
5,xenasolo,0,30629200,0.097
6,cbsnews,1,24747400,-0.254
7,world247breakingnews,0,24664271,0.000
8,abc15arizona,1,23950384,0.045
9,matthewcassel,0,23114400,-0.331


In [89]:
VerifiedAccounts=sortedusers.loc[sortedusers['VerifiedAccount'] == 1]
VerifiedAccounts

,userName,VerifiedAccount,Streams,Score
1,viceworldnews,1,41258500,-0.263
3,skynews,1,34132375,-0.173
6,cbsnews,1,24747400,-0.254
8,abc15arizona,1,23950384,0.045
11,planefinder,1,19829600,-0.173
18,cbsmornings,1,12221719,-0.198


In [90]:
VerifiedAccounts["Score"].mean()

-0.1693333333333333

In [91]:
VerifiedAccounts["Score"].max()

0.045

In [92]:
NoVerifiedAccounts=sortedusers.loc[sortedusers['VerifiedAccount'] == 0]
NoVerifiedAccounts

,userName,VerifiedAccount,Streams,Score
0,president.putin.fanpage,0,48940200,0.114
2,bikini.bottom.news,0,37147500,-0.053
4,cheddygrace,0,31600000,-0.511
5,xenasolo,0,30629200,0.097
7,world247breakingnews,0,24664271,0.000
9,matthewcassel,0,23114400,-0.331
10,aaronparnas6,0,20800000,0.000
12,mihhail83,0,17416100,0.000
13,military_thingz,0,16800000,-0.100
14,visegrad24,0,16694300,0.056


In [93]:
NoVerifiedAccounts["Score"].mean()

-0.047999999999999994

In [94]:
NoVerifiedAccounts["Score"].max()

0.122

In [ ]:
#Bajo este análisis , los perfiles verficados condenan de forma más clara las acciones a través de su videos (descripciones)
#mientras que los no verificados no siguen un perfil tan claro, habiendo mayor presencia de valores positivos 

In [110]:
#Study the relationships and recommendations that the algorithm gives us from each of this profiles, to see if there's any
#common account to most of them
#Graph of the relations
query = """
WITH ['president.putin.fanpage','viceworldnews','bikini.bottom.news','skynews','cheddygrace','xenasolo','cbsnews','world247breakingnews','abc15arizona','matthewcassel','aaronparnas6','planefinder','mihhail83','military_thingz','visegrad24','random.content99','politicsoftheworld','aviationhxb','cbsmornings','city__resident'] as users
 MATCH (u:user)-[r:recommends]->(a:user)
 WHERE u.id in users
 RETURN *
"""
graph.run(query);


In [ ]:
#List of the names of the recommended accounts
query = """
WITH ['president.putin.fanpage','viceworldnews','bikini.bottom.news','skynews','cheddygrace','xenasolo','cbsnews','world247breakingnews','abc15arizona','matthewcassel','aaronparnas6','planefinder','mihhail83','military_thingz','visegrad24','random.content99','politicsoftheworld','aviationhxb','cbsmornings','city__resident'] as users
 MATCH (u:user)-[r:recommends]->(a:user)
 WHERE u.id in users 
RETURN a.id
"""
graph.run(query);

#There are 595

In [ ]:
#I'm not able to thing how to filter or count the number of relations entering a node just taking into account the u being from 
# our list